분할 및 전처리

In [ ]:
# 데이터 처리 및 분석
import pandas as pd
import numpy as np

# 데이터 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 문자
import re

In [ ]:
import pandas as pd

df = pd.read_csv(r'C:\Users\user\Desktop\프로젝트 기획서\SeSac-2024\data_file\Labor_datacombined.csv', encoding='utf-8')
print(df.info())

In [ ]:
import pandas as pd

# 한글 깨짐 방지
plt.rcParams['font.family'] = "Malgun Gothic"

# 마이너스 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# '근로' 만 추출 , 전체 데이터 중 5000개만 사용

import pandas as pd

# 예시로 df 데이터프레임을 생성합니다. 실제로는 이미 존재하는 df를 사용하세요.
# df = pd.read_csv('your_file.csv')

# '근로' 단어가 포함된 행 필터링
df_labor = df.iloc[:10000][df['사건명'].str.contains('근로', na=False) | df['판례내용'].str.contains('근로', na=False)]

# 결과 확인
print(df_labor.tail())

분석 처리

In [ ]:
# 데이터 처리 및 분석
import pandas as pd
import numpy as np

# 데이터 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 문자
import re

In [1]:
import pandas as pd

# 한글 깨짐 방지
plt.rcParams['font.family'] = "Malgun Gothic"

# 마이너스 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

NameError: name 'plt' is not defined

In [ ]:
# '근로' 만 추출 , 전체 데이터 중 5000개만 사용

import pandas as pd

# 예시로 df 데이터프레임을 생성합니다. 실제로는 이미 존재하는 df를 사용하세요.
# df = pd.read_csv('your_file.csv')

# '근로' 단어가 포함된 행 필터링
df_labor = df.iloc[:10000][df['사건명'].str.contains('근로', na=False) | df['판례내용'].str.contains('근로', na=False)]

# 결과 확인
print(df_labor.tail())

### 텍스트 전처리(완) -> 형태소 분석 -> 불용어 제거 -> 명사추출 -> 주제모델링

In [ ]:
import pandas as pd
from collections import Counter
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer

형태소 분석, 불용어제거

In [ ]:
import pandas as pd
import re  # 정규 표현식 모듈
from konlpy.tag import Okt

# 불용어 파일 로드
stopwords_file_path = r'C:\Users\user\Desktop\프로젝트 기획서\SeSac-2024\data_file\sorted_words.txt'
stopwords_pick = ['제', '호로', '등', '것', '항', '의', '금','검사','판결','거나', '유1', '상고이유', '기초', '사실', '기초사실','사고','피고','원고','피고인','사건','담당','변호사','변호인','변론','종결','소외']
pattern = r'제\d+|\d호증|\d호'   # 정규 표현식 패턴

# 불용어 파일을 읽어 불용어 목록 생성
with open(stopwords_file_path, 'r', encoding='utf-8') as f:
    stopwords = set(f.read().splitlines())

# 불용어 제거 함수 (리스트 상태로 처리)
def remove_stopwords(tokens):
    filtered_words = [
        word for word in tokens 
        if len(word) > 1 
        and word not in stopwords 
        and not word.isdigit()  # 숫자인 경우 제외
        and word not in stopwords_pick 
        and not re.match(pattern, word)
    ]
    return ' '.join(filtered_words)  # 문자열로 반환 / 리스트로 변환하지 말 것

# 불용어 제거 수행
df_labor['정제_판례내용'] = df_labor['형태소_토큰'].apply(remove_stopwords)

# 결과 확인
print(df_labor['정제_판례내용'])

명사추출

In [ ]:
okt = Okt()

# 명사 추출 함수
def extract_nouns(text):
    nouns = okt.nouns(text)  # 텍스트에서 명사 추출
    return nouns  # 명사 리스트 반환

# 정제된 판례 내용에서 명사 추출 수행
df_labor['명사_추출'] = df_labor['정제_판례내용'].apply(extract_nouns)

df_labor['명사_추출']

In [ ]:
# 명사별 빈도 확인

# Okt 객체 생성
okt = Okt()

# 명사 추출 함수
def extract_nouns(text):
    nouns = okt.nouns(text)  # 텍스트에서 명사 추출
    return nouns  # 명사 리스트 반환

# 모든 명사 추출 (리스트 평탄화)
all_nouns = [noun for sublist in df_labor['명사_추출'] for noun in sublist]

# 단어 빈도 계산
word_freq = Counter(all_nouns)

# 결과를 데이터프레임으로 변환
freq_df = pd.DataFrame(word_freq.items(), columns=['단어', '빈도']).sort_values(by='빈도', ascending=False)

# 결과 확인
print(freq_df)

In [ ]:
# 상위 100개 단어 빈도 출력
top_100_freq = freq_df.head(100)

# 결과 확인
print(top_100_freq)

#freq_df.iloc[0:100]


주제모델링

In [ ]:
# 명사 추출 함수
def extract_nouns(text):
    nouns = okt.nouns(text)  # 텍스트에서 명사 추출
    return ' '.join(nouns)  # 리스트를 공백으로 구분된 문자열로 반환

# 정제된 판례 내용에서 명사 추출 수행
df_labor['명사_추출2'] = df_labor['정제_판례내용'].apply(extract_nouns)

# LDA를 위한 Count Vectorization
count_vectorizer = CountVectorizer()

# 결측치가 있는 경우 빈 문자열로 처리
count_data = count_vectorizer.fit_transform(df_labor['명사_추출2'].str.lower())

# LDA 모델 생성
num_topics = 5  # 주제 개수 설정
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(count_data)

# 주제 확인
for idx, topic in enumerate(lda.components_):
    print(f"주제 {idx}: ", end='')
    # 각 주제에서 가장 높은 확률을 가진 단어 5개 출력
    print([count_vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-5:]])